In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

mpl.rc('font', family = 'Malgun Gothic')
mpl.rcParams['axes.unicode_minus'] = False

# Day6 복습

In [2]:
df2 = pd.read_csv('./data/09_Data.csv')
df2

,Unnamed: 0.1,Unnamed: 0,date_clean,tag,tag_num,unit,Y1A,Y2A,Y3A,Datetime_x,Datetime_T,Datetime_y,A1,A2,A3,A4,A5,A6
0,0,0,2018-12-01 03:30:00.000,NE280V,GO1701,Total [1/m²],4954.774,327.416,256.882,2018-12-01 03:30:00.000,2018-12-01 03:30:00,2018-12-01 03:30:00,224.686050,227.440741,229.780665,229.752830,230.934634,229.514491
1,1,1,2018-12-01 07:30:00.005,NE280PV,GO1701,Total [1/m²],2281.460,44.410,17.416,2018-12-01 07:30:00.005,2018-12-01 07:30:00,2018-12-01 07:30:00,225.881809,228.579524,230.934893,231.026156,231.873045,230.441799
2,2,2,2018-12-01 11:30:00.010,NE280V,GO1701,Total [1/m²],4453.201,74.017,23.511,2018-12-01 11:30:00.010,2018-12-01 11:30:00,2018-12-01 11:30:00,228.100989,229.933238,231.071409,230.426619,230.495147,228.806310
3,3,3,2018-12-01 15:30:00.015,NE280PV,GO1701,Total [1/m²],4352.190,77.500,37.444,2018-12-01 15:30:00.015,2018-12-01 15:30:00,2018-12-01 15:30:00,228.391907,229.853701,230.969399,230.321498,230.188089,228.399797
4,4,4,2018-12-01 19:30:00.020,NE280V,GO1701,Total [1/m²],4345.224,62.697,40.927,2018-12-01 19:30:00.020,2018-12-01 19:30:00,2018-12-01 19:30:00,229.209428,230.289857,230.993988,230.148035,229.526814,227.565878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,468,468,2019-09-03 13:30:00.000,E280PV,1701,Total [1/m²],22569.040,2422.528,1950.561,2019-09-03 13:30:00.000,2019-09-03 13:30:00,2019-09-03 13:30:00,229.186977,230.492377,231.035415,230.684575,229.567130,227.669265
469,469,469,2019-09-03 15:30:00.000,NE282PV,1701,Total [1/m²],27633.534,3436.123,2903.202,2019-09-03 15:30:00.000,2019-09-03 15:30:00,2019-09-03 15:30:00,229.099485,230.636280,231.010335,230.882376,229.776926,227.896311
470,470,470,2019-09-03 18:30:00.000,NE282PV,1701,Total [1/m²],14115.267,951.667,728.541,2019-09-03 18:30:00.000,2019-09-03 18:30:00,2019-09-03 18:30:00,229.063797,230.467348,230.982656,230.831959,229.723372,227.818912
471,471,471,2019-09-03 19:30:00.000,NE282PV,1701,Total [1/m²],13207.919,788.144,605.793,2019-09-03 19:30:00.000,2019-09-03 19:30:00,2019-09-03 19:30:00,228.958204,230.389537,230.895751,230.572029,229.561304,227.605890


In [3]:
# 9. 'Y1A'값의 상위 90%에 해당하는 값을 1 / 미만을 0로 'Target'이름의 변수로 선언하여
#    'A1'부터 'A6'까지 값을 넣었을때, 'Target'값을 분류하는 분류모델을 생성 
#     - 학습 데이터와 검증데이터 8:2 
#     - 특성공학 기법 -> 결측값 처리 Imputation + 숫자 MinMaxScaling / 교차검증 3회 / 하이퍼파라미터 X 
#     - classification_report 함수로 평가

df2['Target'] = 0
df2.loc[df2['Y1A'] >= df2['Y1A'].quantile(0.9), 'Target'] = 1
df2['Target'].value_counts()

0    425
1     48
Name: Target, dtype: int64

In [4]:
X = df2[['A1','A2','A3','A4','A5','A6']]
Y = df2['Target']

In [5]:
# 불균형 데이터를 위한 라이브러리를 사용할 때는
# imlbearn 에서의 pipeline을 사용해야만 한다 !!!!
from sklearn.model_selection import train_test_split
from imblearn.pipeline import make_pipeline # 불균형 데이터를 위한 라이브러리 -> 에서의 pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE # 오버샘플링의 SMOTE 기법
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2, random_state=42)

# 결측치 -> 스케일링 -> 오버샘플링 -> 학습
model_pipe = make_pipeline(SimpleImputer(strategy='median'), RobustScaler(),
                            SMOTE(), DecisionTreeClassifier())

In [7]:
# 교차검증 & 하이퍼파라미터 튜닝
grid_model = GridSearchCV(model_pipe, cv = 5, param_grid = { },
                         n_jobs = -1, scoring = 'f1')
grid_model.fit(x_train,y_train)
best_model = grid_model.best_estimator_

In [8]:
# 평가
def evaluation_classifier(model):
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)
    
    report_train = classification_report(y_train, y_train_pred, output_dict=True)
    res_train = pd.DataFrame(report_train).T
    
    report_test = classification_report(y_test, y_test_pred, output_dict=True)
    res_test = pd.DataFrame(report_test).T
    display(res_train)
    display(res_test)

evaluation_classifier(best_model)

,precision,recall,f1-score,support
0,0.991404,1.000000,0.995683,346.000000
1,1.000000,0.906250,0.950820,32.000000
accuracy,0.992063,0.992063,0.992063,0.992063
macro avg,0.995702,0.953125,0.973252,378.000000
weighted avg,0.992132,0.992063,0.991885,378.000000


,precision,recall,f1-score,support
0,0.896104,0.873418,0.884615,79.000000
1,0.444444,0.500000,0.470588,16.000000
accuracy,0.810526,0.810526,0.810526,0.810526
macro avg,0.670274,0.686709,0.677602,95.000000
weighted avg,0.820035,0.810526,0.814884,95.000000


---

# 알고리즘

# 홈쇼핑 데이터

In [9]:
pd.options.display.max_columns = 30

In [10]:
df = pd.read_csv('./data/12_Data.csv')
df

,방송Code,채널,소요분,가중분,방송구분,프로그램명,상품ID,상품명,매입과세구분,상품목표취급금액,상품목표주문금액,판매단가,수수료율,상품주문수량,상품주문금액,상품취소수량,상품취소금액,ARS금액,매입형태,배송방식,상품소요분,상품가중분,상품방송순번,방송시작시간,방송종료시간
0,1.010036e+09,TV,50.0,52.7,녹화방송,재방_의류,10242138,[비지트인뉴욕콜렉션] 퓨어 캐시미어 롱코트,과세,65645345,117223831,149000,25.0,648,86284484,73,9624136,1000,위탁매입,협력사배송,50.0,52.7,1,2020-01-03T00:10,2020-01-03T01:00
1,1.019126e+09,TV,60.0,38.1,녹화방송,재방_건강식품,10092003,[한삼인]순홍삼진(50ml*30포)*7박스+쇼7_2,과세,23774849,29178755,179000,26.0,200,32219095,16,2547305,1000,위탁매입,직택배,30.0,19.1,1,2020-01-03T01:00,2020-01-03T02:00
2,1.019226e+09,TV,60.0,38.1,녹화방송,재방_건강식품,10092743,[단품_한삼인] 순홍삼진(50ml*30포)*1박스,과세,23774849,29178755,60000,26.0,0,0,0,0,0,위탁매입,직택배,30.0,19.1,2,2020-01-03T01:00,2020-01-03T02:00
3,1.040337e+09,TV,60.0,19.7,재방송,재방_가공농산,10295865,이상용의 우리밀 크레마롤,과세,24568911,28904601,48900,24.0,460,19860828,25,1080206,1000,위탁매입,협력사배송,60.0,19.7,1,2020-01-03T02:00,2020-01-03T03:00
4,1.021427e+09,TV,60.0,16.1,재방송,재방_신선수산,10113666,국내산 자숙 문어슬라이스 120g*9팩,면세,20131383,24550467,49900,22.0,125,6090566,5,238512,1000,위탁매입,협력사배송,60.0,16.1,1,2020-01-03T03:00,2020-01-03T04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26278,1.046263e+13,TV,60.0,94.3,생방송,가전,11179512,■연말특가■제우스커브드 UHD TV 65형,과세,92367433,151422021,649000,15.0,243,143118201,20,11783638,1000,위탁매입,협력사배송,19.9,32.6,1,2021-11-13T21:45,2021-11-13T22:45
26279,1.046263e+13,TV,60.0,94.3,생방송,가전,11179514,2019 제우스 UHD TV 75형,과세,33698544,55243515,1289000,15.0,76,88954537,4,4683636,1000,위탁매입,협력사배송,26.7,41.9,2,2021-11-13T21:45,2021-11-13T22:45
26280,1.046263e+13,TV,60.0,94.3,생방송,가전,11179504,■연말특가■제우스커브드 UHD TV 55형,과세,17202608,28200996,449000,15.0,108,43990026,9,3667275,1000,위탁매입,협력사배송,13.4,22.7,3,2021-11-13T21:45,2021-11-13T22:45
26281,1.020263e+13,TV,60.0,91.6,재방송,재방_건강식품,11152474,[래오이경제] 흑염소진액 70ml x 120포,과세,154250864,202961664,159000,30.0,1363,195595043,27,3879990,1000,위탁매입,협력사배송,60.0,91.6,1,2021-11-13T22:45,2021-11-13T23:45


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26283 entries, 0 to 26282
Data columns (total 25 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   방송Code    26283 non-null  float64
 1   채널        26283 non-null  object 
 2   소요분       26283 non-null  float64
 3   가중분       26283 non-null  float64
 4   방송구분      26283 non-null  object 
 5   프로그램명     26283 non-null  object 
 6   상품ID      26283 non-null  int64  
 7   상품명       26283 non-null  object 
 8   매입과세구분    26283 non-null  object 
 9   상품목표취급금액  26283 non-null  int64  
 10  상품목표주문금액  26283 non-null  int64  
 11  판매단가      26283 non-null  int64  
 12  수수료율      26283 non-null  float64
 13  상품주문수량    26283 non-null  int64  
 14  상품주문금액    26283 non-null  int64  
 15  상품취소수량    26283 non-null  int64  
 16  상품취소금액    26283 non-null  int64  
 17  ARS금액     26283 non-null  int64  
 18  매입형태      26283 non-null  object 
 19  배송방식      26283 non-null  object 
 20  상품소요분     26283 non-null  fl

##### 선형회귀 모델을 이용한 분석

In [12]:
df1 = df[['상품목표주문금액','판매단가','상품목표취급금액','수수료율','ARS금액','소요분',
    '상품주문금액']]

In [13]:
# 결측치 확인
df1.isnull().sum()

상품목표주문금액    0
판매단가        0
상품목표취급금액    0
수수료율        0
ARS금액       0
소요분         0
상품주문금액      0
dtype: int64

In [14]:
X = df1[['상품목표주문금액','판매단가','상품목표취급금액','수수료율','ARS금액','소요분']]
Y = df1['상품주문금액']

In [15]:
# 선형회귀 알고리즘을 가져와서 모델링
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [16]:
x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size = 0.3, random_state=42)

In [17]:
model = LinearRegression()
model.fit(x_train,y_train)

LinearRegression()

In [18]:
# 회귀계수
model.coef_

array([ 1.04513943e+00,  5.16777635e+00, -1.36280347e-01, -9.96757106e+05,
       -1.98020209e+02,  2.49535055e+05])

In [19]:
# 회귀 절편
model.intercept_

4670704.287531957

    Y(상품주문금액) = 1.045x(상품목표주문금액) + 5.178x(판매단가) -0.136x(상품목표취급금액)
                      - 996757x(수수료율) - 198x(ARS금액) + 249535x(소요분) + 4670677.50

In [20]:
# 규제 선형 회귀 모델
from sklearn.linear_model import Ridge

In [21]:
model = Ridge()
model.fit(x_train,y_train)
model.coef_

array([ 1.04513952e+00,  5.16777913e+00, -1.36280434e-01, -9.96755238e+05,
       -1.98020865e+02,  2.49534754e+05])

In [22]:
from sklearn.linear_model import Lasso

In [23]:
model = Lasso()
model.fit(x_train,y_train)
model.coef_

C:\Users\admin\anaconda3\envs\python3112\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.896e+18, tolerance: 1.499e+16
  model = cd_fast.enet_coordinate_descent(


array([ 1.04513943e+00,  5.16777643e+00, -1.36280348e-01, -9.96757069e+05,
       -1.98020223e+02,  2.49535042e+05])

---

In [24]:
df3 = pd.read_csv('./data/09_Data.csv').dropna()
df3[['A1','A2','A3','A4','A5','A6']]
df3['Y1A']

0       4954.774
1       2281.460
2       4453.201
3       4352.190
4       4345.224
         ...    
468    22569.040
469    27633.534
470    14115.267
471    13207.919
472    15553.431
Name: Y1A, Length: 448, dtype: float64

In [25]:
model1 = LinearRegression()
model2 = Lasso(alpha = 1000)

model1.fit(X,Y)
model2.fit(X,Y)

print(model1.coef_)
print(model2.coef_)

[ 1.04462195e+00  5.38232711e+00 -1.37154611e-01 -1.04061083e+06
 -3.03665968e+02  2.13269112e+05]
[ 1.04462390e+00  5.38239851e+00 -1.37155969e-01 -1.04057399e+06
 -3.03679832e+02  2.13256330e+05]


---

# Mission 9
- 12_Data.csv를 불러와 df1으로 선언하여 아래의 미션을 수행

In [26]:
df1 = pd.read_csv('./data/12_Data.csv')

In [27]:
# 1. '방송구분'이라는 항목에서 '녹화방송'에 해당하는 데이터의 '프로그램명'별 '상품목표주문금액'과 '상품주문금액'의 합과 평균을 계산
df1.loc[df1['방송구분'] == '녹화방송'].pivot_table(index='프로그램명',
                values = ['상품목표주문금액', '상품주문금액'], aggfunc = ['sum','mean'])

sum                     mean              
           상품목표주문금액     상품주문금액      상품목표주문금액        상품주문금액
프로그램명                                                     
게릴라 프로모션    6386241    8736100  6.386241e+06  8.736100e+06
재방_가전      30944319   41860280  1.031477e+07  1.395343e+07
재방_건강식품    97042110   56468669  2.426053e+07  1.411717e+07
재방_생활용품    31734436   38868330  6.346887e+06  7.773666e+06
재방_속옷      75912426   36617333  3.795621e+07  1.830867e+07
재방_신선농산    86213568   74268755  2.155339e+07  1.856719e+07
재방_의류     471714526  372461954  6.738779e+07  5.320885e+07
재방_이미용     75910061   64767805  1.084429e+07  9.252544e+06
재방_잡화      32719026   32174842  1.635951e+07  1.608742e+07
재방_침구      54728388   47626117  1.368210e+07  1.190653e+07

In [28]:
# 2. '프로그램명'에서 가장'상품주문금액' 합이 큰 3가지를 확인하여, 3가지 항목에 대해 데이터를 추출한 뒤
# '상품명'별 '상품주문금액'의 합을 계산하시오.

top_list = df1.pivot_table(index = '프로그램명', values = '상품주문금액',
                aggfunc = 'sum').sort_values(by='상품주문금액', ascending = False).head(3).index
df1.loc[df1['프로그램명'].isin(top_list)].pivot_table(index = '상품명', values = '상품주문금액', aggfunc = 'sum')

,상품주문금액
상품명,
모닝애플 조생부사 세척사과 3kg*3박스,83710050
모닝애플 홍로 세척사과 3kg*3박스,260894995
(17년 햅곡)김연도 오색현미 + 옹기가마솥 세트,113245285
(17년 햅곡)김연도 오색현미 22봉,61467733
(17년 햅곡)김연도 웰빙 선물세트,40103888
...,...
황금대추 방울토마토3kg,153834944
황토방 청도 감말랭이 세트(30봉),41407760
황토방 청도 감말랭이 세트(30봉+1봉),106544036


In [29]:
# 3. '방송시작시간'과 '방송종료시간'의 차이를 계산하여, '방송진행시간'이라는 항목을 선언
pd.to_datetime(df1['방송시작시간'])
pd.to_datetime(df1['방송종료시간'])

df1['방송진행시간'] = pd.to_datetime(df1['방송종료시간']) - pd.to_datetime(df1['방송시작시간'])
df1

,방송Code,채널,소요분,가중분,방송구분,프로그램명,상품ID,상품명,매입과세구분,상품목표취급금액,상품목표주문금액,판매단가,수수료율,상품주문수량,상품주문금액,상품취소수량,상품취소금액,ARS금액,매입형태,배송방식,상품소요분,상품가중분,상품방송순번,방송시작시간,방송종료시간,방송진행시간
0,1.010036e+09,TV,50.0,52.7,녹화방송,재방_의류,10242138,[비지트인뉴욕콜렉션] 퓨어 캐시미어 롱코트,과세,65645345,117223831,149000,25.0,648,86284484,73,9624136,1000,위탁매입,협력사배송,50.0,52.7,1,2020-01-03T00:10,2020-01-03T01:00,0 days 00:50:00
1,1.019126e+09,TV,60.0,38.1,녹화방송,재방_건강식품,10092003,[한삼인]순홍삼진(50ml*30포)*7박스+쇼7_2,과세,23774849,29178755,179000,26.0,200,32219095,16,2547305,1000,위탁매입,직택배,30.0,19.1,1,2020-01-03T01:00,2020-01-03T02:00,0 days 01:00:00
2,1.019226e+09,TV,60.0,38.1,녹화방송,재방_건강식품,10092743,[단품_한삼인] 순홍삼진(50ml*30포)*1박스,과세,23774849,29178755,60000,26.0,0,0,0,0,0,위탁매입,직택배,30.0,19.1,2,2020-01-03T01:00,2020-01-03T02:00,0 days 01:00:00
3,1.040337e+09,TV,60.0,19.7,재방송,재방_가공농산,10295865,이상용의 우리밀 크레마롤,과세,24568911,28904601,48900,24.0,460,19860828,25,1080206,1000,위탁매입,협력사배송,60.0,19.7,1,2020-01-03T02:00,2020-01-03T03:00,0 days 01:00:00
4,1.021427e+09,TV,60.0,16.1,재방송,재방_신선수산,10113666,국내산 자숙 문어슬라이스 120g*9팩,면세,20131383,24550467,49900,22.0,125,6090566,5,238512,1000,위탁매입,협력사배송,60.0,16.1,1,2020-01-03T03:00,2020-01-03T04:00,0 days 01:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26278,1.046263e+13,TV,60.0,94.3,생방송,가전,11179512,■연말특가■제우스커브드 UHD TV 65형,과세,92367433,151422021,649000,15.0,243,143118201,20,11783638,1000,위탁매입,협력사배송,19.9,32.6,1,2021-11-13T21:45,2021-11-13T22:45,0 days 01:00:00
26279,1.046263e+13,TV,60.0,94.3,생방송,가전,11179514,2019 제우스 UHD TV 75형,과세,33698544,55243515,1289000,15.0,76,88954537,4,4683636,1000,위탁매입,협력사배송,26.7,41.9,2,2021-11-13T21:45,2021-11-13T22:45,0 days 01:00:00
26280,1.046263e+13,TV,60.0,94.3,생방송,가전,11179504,■연말특가■제우스커브드 UHD TV 55형,과세,17202608,28200996,449000,15.0,108,43990026,9,3667275,1000,위탁매입,협력사배송,13.4,22.7,3,2021-11-13T21:45,2021-11-13T22:45,0 days 01:00:00
26281,1.020263e+13,TV,60.0,91.6,재방송,재방_건강식품,11152474,[래오이경제] 흑염소진액 70ml x 120포,과세,154250864,202961664,159000,30.0,1363,195595043,27,3879990,1000,위탁매입,협력사배송,60.0,91.6,1,2021-11-13T22:45,2021-11-13T23:45,0 days 01:00:00


In [30]:
# 4. '방송시작시간'을 날짜형식으로 변환하여, '방송연도'/'방송월'/'방송일'/'방송요일'항목을 생성
df1['방송연도'] = pd.to_datetime(df1['방송시작시간']).dt.year
df1['방송월'] = pd.to_datetime(df1['방송시작시간']).dt.month
df1['방송일'] = pd.to_datetime(df1['방송시작시간']).dt.day
df1['방송요일'] = pd.to_datetime(df1['방송시작시간']).dt.day_name()
df1[['방송연도','방송월','방송일','방송요일']]

,방송연도,방송월,방송일,방송요일
0,2020,1,3,Friday
1,2020,1,3,Friday
2,2020,1,3,Friday
3,2020,1,3,Friday
4,2020,1,3,Friday
...,...,...,...,...
26278,2021,11,13,Saturday
26279,2021,11,13,Saturday
26280,2021,11,13,Saturday
26281,2021,11,13,Saturday


In [31]:
# 5. '방송요일'항목에서 주말과 주중을 구분하여, '주말'과 '주중'의 '상품주문수량'과 '상품주문금액'의 평균의 차이를 계산 
weekday = ['Monday','Tuesday','Wednesday','Thursday','Friday']
weekend = ['Saturday','Sunday']
df1.loc[df1['방송요일'].isin(weekday), '주말주중'] = '주중'
df1.loc[df1['방송요일'].isin(weekend), '주말주중'] = '주말'
pivot_q5 = df1.pivot_table(index ='주말주중', values = ['상품주문수량','상품주문금액'], aggfunc = 'mean')
a = pivot_q5['상품주문금액']['주말'] - pivot_q5['상품주문금액']['주중']
b = pivot_q5['상품주문수량']['주말'] - pivot_q5['상품주문수량']['주중']
print('상품주문금액의 주말과 주중의 평균차이 :',a,'원')
print('상품주문수량의 주말과 주중의 평균차이 :',b,'개')

상품주문금액의 주말과 주중의 평균차이 : -11246180.77671355 원
상품주문수량의 주말과 주중의 평균차이 : 16.6918846481193 개


In [32]:
# # 5번 문제 간단히 !!!
day_of_week_list = df1['방송요일'].unique().tolist()
print(day_of_week_list)
weekend_list = ['주중','주말','주말','주중','주중','주중','주중']
df1['주말주중'] = df1['방송요일'].replace(dict(zip(day_of_week_list, weekend_list)))
df1[['주말주중']]

['Friday', 'Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday']


,주말주중
0,주중
1,주중
2,주중
3,주중
4,주중
...,...
26278,주말
26279,주말
26280,주말
26281,주말


In [33]:
# 6. '배송방식'과 '매입형태'간의 연관성이 있는지 확인하고자 한다. 
#     - Contingency Table을 계산하고, 두 항목간 독립성 검정을 수행 
cross_q6 = pd.crosstab(df1['배송방식'],df1['매입형태'])
display(cross_q6)

# H0 : 두 데이터는 서로 독립이다.(연관성이 없다.)
# H1 : 두 데이터는 서로 독립이 아니다.(연관성이 있다.)

stats.chi2_contingency(cross_q6)
# p-value < 0.05 이므로 H0 기각
# 따라서 두 데이터는 서로 연관이 있다고 볼 수 있다.

매입형태,위탁매입,직매입
배송방식,,
당사배송,1,1019
직택배,996,0
협력사배송,24267,0


(26256.193038804344,
 0.0,
 2,
 array([[  980.45428604,    39.54571396],
        [  957.38477343,    38.61522657],
        [23326.16094053,   940.83905947]]))

In [34]:
# 7. '방송구분'에서 '생방송'과 '녹화방송'의 매출을 비교하고자 한다. '생방송'과'녹화방송'의 '상품주문금액'의 평균의 차이가 있는지 검정 
# 1) 정규성 검정
# H0 : 상품주문금액의 분포는 정규성을 따른다.
# H1 : 상품주문금액의 분포는 정규성을 따르지 않는다.
df1_생방녹방 = df1.loc[df1['방송구분'].isin(['생방송','녹화방송'])]
stats.normaltest(df1_생방녹방['상품주문금액'])
# p-value < 0.05 이므로 H0기각
# 따라서 정규성을 따르지 않는다.

NormaltestResult(statistic=17972.62409951878, pvalue=0.0)

In [35]:
# 2) 평균 검정
# 정규성을 따르지 않으므로 윌콕슨 순위합 검정 실시
# H0 : 두 데이터의 평균 차이는 없다.
# H1 : 두 데이터의 평균 차이는 있다.

df1_생방송 = df1.loc[df1['방송구분'] == '생방송']
df1_녹화방송 = df1.loc[df1['방송구분'] == '녹화방송']
print('p-value 값 :',stats.ranksums(df1_생방송['상품주문금액'], df1_녹화방송['상품주문금액'])[1])
# p-value < 0.05이므로 H0 기각
# 따라서 두 데이터의 평균 차이는 있다고 볼 수 있다.

a = df1_생방송['상품주문금액'].mean()
b = df1_녹화방송['상품주문금액'].mean()

print(f'생방송의 상품주문금액 평균 : {a}')
print(f'녹화방송의 상품주문금액 평균 : {b}')

p-value 값 : 3.698970792035826e-09
생방송의 상품주문금액 평균 : 82167318.82283759
녹화방송의 상품주문금액 평균 : 19842312.435897436


In [36]:
# 8. '상품목표주문금액'에서 '상품주문금액'의 차이를 계산해
# 목표를 달성하지 못한 방송에 대해서는 0 / 목표를 달성한 방송은 1값으로 '목표달성여부' 변수를 생성
df1.loc[df1['상품주문금액'] - df1['상품목표주문금액'] > 0, '목표달성여부'] = 1
df1.loc[df1['상품주문금액'] - df1['상품목표주문금액'] < 0, '목표달성여부'] = 0
df1[['목표달성여부']].value_counts()

# 목표를 반 이상 달성하지 못함
# 목표를 달성할 수 있음에도 달성하지 못한 이유가 있거나
# 목표를 너무 크게 잡았거나 등등의 이유가 있을 수 있음
# 그 이유는 다른 요인들의 분석을 통해 확인

목표달성여부
0.0       16333
1.0        9323
dtype: int64

In [37]:
# 9. '목표달성여부'를 분류하는 모델을 만들고자 한다.
#     - X : 소요분 / 방송구분 / 판매단가 / ARS 금액 /수수료율 / '방송요일' / '방송월'
#     - Y : '목표달성여부'
#     - 학습 : 검증 = 8 : 2
#     - 특성 공학 기법 (결측처리(중앙값,최빈값) + 스케일링&인코딩 +교차검증3회)
#     - 알고리즘 (Decision Tree 알고리즘 / 하이퍼파라미터 튜닝) 
#     - 평가 
XY = df1[['소요분','방송구분','판매단가','ARS금액','수수료율','방송요일','방송월','목표달성여부']].dropna()
X = XY[['소요분','방송구분','판매단가','ARS금액','수수료율','방송요일','방송월']]
Y = XY['목표달성여부']

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2, random_state=42)

from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

num_pipe = make_pipeline(SimpleImputer(strategy='median'),MinMaxScaler())
cat_pipe = make_pipeline(SimpleImputer(strategy='most_frequent'),OneHotEncoder(handle_unknown='ignore'))
# OneHotEncoder(handle_unknown='ignore') : 새로운 모르는 값이 왔을 경우 무시

num_list = X.select_dtypes(exclude = 'object').columns
cat_list = X.select_dtypes(include = 'object').columns

preprocessing_pipe = make_column_transformer((num_pipe, num_list),
                                             (cat_pipe, cat_list))

model_pipe = make_pipeline(preprocessing_pipe,DecisionTreeClassifier())
model_pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  Index(['소요분', '판매단가', 'ARS금액', '수수료율', '방송월'], dtype='object')),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['방송구분', '방송요일'], dtype='object'))])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [38]:
# 하이퍼 파라미터 튜닝

# hyper_param = {
#               'decisiontreeclassifier__criterion' : ['gini','entropy'],
#               'decisiontreeclassifier__max_depth' : range(5,15),
#               'decisiontreeclassifier__min_samples_split' : range(10,20)
#               }

grid_model = GridSearchCV(model_pipe, param_grid= {}, cv = 3,
                          scoring = 'f1',
                         n_jobs = -1)
grid_model.fit(x_train,y_train)

best_model = grid_model.best_estimator_
best_model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  Index(['소요분', '판매단가', 'ARS금액', '수수료율', '방송월'], dtype='object')),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['방송구분', '방송요일'], dtype='object'))])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [39]:
def evaluation_classifier(model):
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)
    
    report_train = classification_report(y_train, y_train_pred, output_dict=True)
    res_train = pd.DataFrame(report_train).T
    
    report_test = classification_report(y_test, y_test_pred, output_dict=True)
    res_test = pd.DataFrame(report_test).T
    display(res_train)
    display(res_test)
    
evaluation_classifier(best_model)

,precision,recall,f1-score,support
0.0,0.947242,0.996412,0.971205,13100.000000
1.0,0.993031,0.902074,0.945370,7424.000000
accuracy,0.962288,0.962288,0.962288,0.962288
macro avg,0.970137,0.949243,0.958288,20524.000000
weighted avg,0.963805,0.962288,0.961860,20524.000000


,precision,recall,f1-score,support
0.0,0.690265,0.699660,0.694931,3233.000000
1.0,0.476550,0.465508,0.470964,1899.000000
accuracy,0.613016,0.613016,0.613016,0.613016
macro avg,0.583408,0.582584,0.582948,5132.000000
weighted avg,0.611184,0.613016,0.612056,5132.000000


##### 11_Data.csv파일을 가져와 df5로 선언하고 아래의 문제를 해결 

In [40]:
df5 = pd.read_csv('./data/11_Data.csv')
df5

,Unnamed: 0,Ox_Chamber,process,type,Temp_OXid,Vapor,ppm,Pressure,Oxid_time,thickness,No_Die,Reinforcement,Unnamed: 0_x,photo_soft_Chamber,process 2,...,Chamber_Num,process4,Flux60s,Flux90s,Flux160s,Flux480s,Flux840s,input_Energy,Temp_implantation,Furance_Temp,RTA_Temp,Target,Error_message,target_binom,Chamber_Route
0,0,1,Oxidation,dry,1138.979159,O2,32.80,0.200,62.0,699.443,NOLSM111,True,0,1,Photo,...,3,Implantation,9.997373e+15,1.346079e+17,3.468281e+17,3.002593e+17,6.000007e+17,31574.410,102.847,885.0,154,96,none,0.0,route_11133
1,1,1,Oxidation,dry,1218.184551,O2,31.86,0.194,137.0,696.792,NOLSM212,True,1,1,Photo,...,2,Implantation,9.705928e+15,1.389326e+17,3.838008e+17,3.017903e+17,6.000012e+17,31580.213,104.323,919.0,154,102,none,0.0,route_11222
2,2,1,Oxidation,dry,1062.467808,O2,39.51,0.217,128.0,705.471,NOLSM313,False,2,1,Photo,...,1,Implantation,1.761358e+16,8.782248e+16,6.014497e+17,2.994231e+17,6.000002e+17,32162.414,100.605,916.0,155,95,none,0.0,route_11311
3,3,1,Oxidation,dry,1114.704773,O2,32.88,0.201,90.0,710.772,NOLSM414,False,3,2,Photo,...,1,Implantation,1.148303e+16,9.776566e+16,6.646884e+16,2.991354e+17,6.000003e+17,32874.925,101.739,911.0,156,117,none,0.0,route_12111
4,4,1,Oxidation,dry,989.411946,O2,38.11,0.204,98.0,716.975,NOLSM515,False,4,2,Photo,...,2,Implantation,9.769204e+15,1.222219e+17,2.496090e+17,3.005576e+17,6.000013e+17,30985.928,106.422,872.0,155,143,none,0.0,route_12222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747,845,3,Oxidation,wet,1280.687973,H2O,45.19,0.214,21.0,708.586,NOLSM846329,False,845,3,Photo,...,1,Implantation,1.813283e+16,1.763143e+17,1.026166e+18,2.995317e+17,5.999986e+17,32252.961,101.177,868.0,152,84,none,0.0,route_33111
748,846,3,Oxidation,wet,1275.153349,H2O,45.08,0.215,22.0,712.936,NOLSM8473210,False,846,3,Photo,...,2,Implantation,2.441931e+15,2.310319e+16,1.359659e+17,3.004926e+17,5.999991e+17,32253.818,100.736,868.0,151,105,none,0.0,route_33222
749,847,3,Oxidation,wet,1275.182502,H2O,45.10,0.214,21.0,715.498,NOLSM8483211,False,847,3,Photo,...,3,Implantation,1.054583e+16,9.513525e+16,6.272553e+17,3.009325e+17,6.000003e+17,32248.621,101.503,868.0,152,78,none,0.0,route_33333
750,848,1,Oxidation,wet,1268.105427,H2O,45.07,0.215,22.0,707.179,NOLSM8493212,False,848,1,Photo,...,3,Implantation,6.909171e+15,5.426149e+16,1.865029e+17,3.006733e+17,6.000003e+17,32241.426,101.061,867.0,152,42,none,0.0,route_11133


In [41]:
df5.columns

Index(['Unnamed: 0', 'Ox_Chamber', 'process', 'type', 'Temp_OXid', 'Vapor',
       'ppm', 'Pressure', 'Oxid_time', 'thickness', 'No_Die', 'Reinforcement',
       'Unnamed: 0_x', 'photo_soft_Chamber', 'process 2', 'resist_target',
       'N2_HMDS', 'pressure_HMDS', 'temp_HMDS', 'temp_HMDS_bake',
       'time_HMDS_bake', 'spin1', 'spin2', 'spin3', 'photoresist_bake',
       'temp_softbake', 'time_softbake', 'lithography_Chamber', 'Line_CD',
       'UV_type', 'Wavelength', 'Resolution', 'Energy_Exposure', 'Range_check',
       'Unnamed: 0_y', 'Etching_Chamber', 'Process 3', 'Temp_Etching',
       'Source_Power', 'Selectivity', 'Thin Film 4', 'Thin Film 3',
       'Thin Film 2', 'Thin Film 1', 'Etching_rate', 'Chamber_Num', 'process4',
       'Flux60s', 'Flux90s', 'Flux160s', 'Flux480s', 'Flux840s',
       'input_Energy', 'Temp_implantation', 'Furance_Temp', 'RTA_Temp',
       'Target', 'Error_message', 'target_binom', 'Chamber_Route'],
      dtype='object')

In [42]:
# 데이터 핸들링 --> chamber 관련 및 필요없는 컬럼 제거
df6 = df5.drop(columns=['Unnamed: 0', 'Ox_Chamber','Unnamed: 0_x','photo_soft_Chamber',
                 'lithography_Chamber','Unnamed: 0_y', 'Etching_Chamber',
                 'Chamber_Num', 'Error_message', 'target_binom', 'Chamber_Route'])
df6

,process,type,Temp_OXid,Vapor,ppm,Pressure,Oxid_time,thickness,No_Die,Reinforcement,process 2,resist_target,N2_HMDS,pressure_HMDS,temp_HMDS,...,Thin Film 3,Thin Film 2,Thin Film 1,Etching_rate,process4,Flux60s,Flux90s,Flux160s,Flux480s,Flux840s,input_Energy,Temp_implantation,Furance_Temp,RTA_Temp,Target
0,Oxidation,dry,1138.979159,O2,32.80,0.200,62.0,699.443,NOLSM111,True,Photo,1.211940,13.891,15.025,20.000,...,267.06,303.82,361.38,2.75950,Implantation,9.997373e+15,1.346079e+17,3.468281e+17,3.002593e+17,6.000007e+17,31574.410,102.847,885.0,154,96
1,Oxidation,dry,1218.184551,O2,31.86,0.194,137.0,696.792,NOLSM212,True,Photo,0.887720,14.722,15.059,19.995,...,332.74,369.31,426.11,2.72775,Implantation,9.705928e+15,1.389326e+17,3.838008e+17,3.017903e+17,6.000012e+17,31580.213,104.323,919.0,154,102
2,Oxidation,dry,1062.467808,O2,39.51,0.217,128.0,705.471,NOLSM313,False,Photo,1.113156,19.205,14.964,19.997,...,424.21,460.65,517.80,2.67000,Implantation,1.761358e+16,8.782248e+16,6.014497e+17,2.994231e+17,6.000002e+17,32162.414,100.605,916.0,155,95
3,Oxidation,dry,1114.704773,O2,32.88,0.201,90.0,710.772,NOLSM414,False,Photo,0.882195,16.368,14.944,19.998,...,272.28,308.56,365.93,2.74825,Implantation,1.148303e+16,9.776566e+16,6.646884e+16,2.991354e+17,6.000003e+17,32874.925,101.739,911.0,156,117
4,Oxidation,dry,989.411946,O2,38.11,0.204,98.0,716.975,NOLSM515,False,Photo,0.834001,13.926,14.933,20.002,...,434.98,471.65,528.85,2.74625,Implantation,9.769204e+15,1.222219e+17,2.496090e+17,3.005576e+17,6.000013e+17,30985.928,106.422,872.0,155,143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747,Oxidation,wet,1280.687973,H2O,45.19,0.214,21.0,708.586,NOLSM846329,False,Photo,0.923802,12.463,15.112,19.996,...,221.44,257.92,314.98,2.67450,Implantation,1.813283e+16,1.763143e+17,1.026166e+18,2.995317e+17,5.999986e+17,32252.961,101.177,868.0,152,84
748,Oxidation,wet,1275.153349,H2O,45.08,0.215,22.0,712.936,NOLSM8473210,False,Photo,0.837348,19.767,14.910,19.999,...,245.20,281.75,339.09,2.72725,Implantation,2.441931e+15,2.310319e+16,1.359659e+17,3.004926e+17,5.999991e+17,32253.818,100.736,868.0,151,105
749,Oxidation,wet,1275.182502,H2O,45.10,0.214,21.0,715.498,NOLSM8483211,False,Photo,0.859869,16.029,15.005,19.998,...,192.10,228.65,285.91,2.72275,Implantation,1.054583e+16,9.513525e+16,6.272553e+17,3.009325e+17,6.000003e+17,32248.621,101.503,868.0,152,78
750,Oxidation,wet,1268.105427,H2O,45.07,0.215,22.0,707.179,NOLSM8493212,False,Photo,0.914315,17.587,14.949,20.000,...,194.33,230.76,287.66,2.69150,Implantation,6.909171e+15,5.426149e+16,1.865029e+17,3.006733e+17,6.000003e+17,32241.426,101.061,867.0,152,42


In [43]:
# 10. 'Target'값을 예측하는 회귀 모델을 구성 
#     - 모든 연속형 데이터를 X로 'Target'을 Y로 선언하여 회귀모델을 구성  
#     - 특성공학 X
#     - Lasso 회귀모델을 이용해, 회귀계수가 0인 변수를 확인
#     - 그리고 유의미한 변수만 추출한다.
num_list = df6.select_dtypes(exclude = 'object').columns.tolist()
X = df6[num_list].drop('Target', axis = 1)
Y = df6['Target']

x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2, random_state=42)

from sklearn.linear_model import Lasso
pipe_model = make_pipeline(MinMaxScaler(), Lasso())
pipe_model.fit(x_train,y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()), ('lasso', Lasso())])

In [44]:
df_coef_lasso = pd.DataFrame()
df_coef_lasso['Coef'] = pipe_model['lasso'].coef_
df_coef_lasso['Feature'] = X.columns
df_coef_lasso

,Coef,Feature
0,-8.950908,Temp_OXid
1,-7.612669,ppm
2,70.411918,Pressure
3,10.653892,Oxid_time
4,1.699612,thickness
5,-7.671954,Reinforcement
6,8.326016,resist_target
7,0.000000,N2_HMDS
8,-0.000000,pressure_HMDS
9,0.000000,temp_HMDS


In [45]:
# 현재 모델의 성능이 좋다는 가정하에 feature들을 뽑아주었음
select_feature_list = df_coef_lasso.loc[df_coef_lasso['Coef'] > 0]['Feature'].tolist()
df7 = df6[select_feature_list]
df7

,Pressure,Oxid_time,thickness,resist_target,Line_CD,Temp_Etching,Thin Film 2,Etching_rate
0,0.200,62.0,699.443,1.211940,30.959,70.878,303.82,2.75950
1,0.194,137.0,696.792,0.887720,29.653,69.561,369.31,2.72775
2,0.217,128.0,705.471,1.113156,28.063,70.968,460.65,2.67000
3,0.201,90.0,710.772,0.882195,31.556,70.146,308.56,2.74825
4,0.204,98.0,716.975,0.834001,31.969,71.174,471.65,2.74625
...,...,...,...,...,...,...,...,...
747,0.214,21.0,708.586,0.923802,35.404,70.859,257.92,2.67450
748,0.215,22.0,712.936,0.837348,31.011,71.294,281.75,2.72725
749,0.214,21.0,715.498,0.859869,32.525,71.550,228.65,2.72275
750,0.215,22.0,707.179,0.914315,28.001,70.718,230.76,2.69150
